In [1]:
from collections import defaultdict
import os
import yaml
import omero.clients
from omero.gateway import BlitzGateway
from omero.rtypes import unwrap

In [2]:
with open(os.path.expanduser('~/blitzgateway-credentials.yml')) as f:
    creds = yaml.safe_load(f)
conn = BlitzGateway(**creds, secure=True)
c = conn.connect()
assert c

In [3]:
qs = conn.c.getSession().getQueryService()
params = omero.sys.ParametersI()

In [4]:
rs = qs.projection("""
    SELECT i.id, i.name
    FROM Project p
    JOIN p.datasetLinks pdl
    JOIN pdl.child ds
    JOIN ds.imageLinks dil
    JOIN dil.child i
    JOIN i.pixels px
    JOIN px.channels c
    JOIN c.logicalChannel lc
    WHERE p.name LIKE 'idr0071%'
    AND lc.name = '0'
""", params)
print(unwrap(rs))

[]


In [5]:
rs = qs.projection("""
    SELECT DISTINCT(lc.name)
    FROM Project p
    JOIN p.datasetLinks pdl
    JOIN pdl.child ds
    JOIN ds.imageLinks dil
    JOIN dil.child i
    JOIN i.pixels px
    JOIN px.channels c
    LEFT OUTER JOIN c.logicalChannel lc
    WHERE p.name LIKE 'idr0071%'
""", params)
print(unwrap(rs))

[['A594'], ['Cy7'], ['Cy5'], ['Cells'], ['DAPI'], ['Cy3'], ['p65'], ['Nuclei']]


In [6]:
rs = unwrap(qs.projection("""
    SELECT i.id, lc.name
    FROM Project p
    JOIN p.datasetLinks pdl
    JOIN pdl.child ds
    JOIN ds.imageLinks dil
    JOIN dil.child i
    JOIN i.pixels px
    JOIN px.channels c
    LEFT OUTER JOIN c.logicalChannel lc
    WHERE p.name LIKE 'idr0071%'
""", params))

image_channel_list = defaultdict(set)
for r in rs:
    image_channel_list[r[0]].add(r[1])

In [7]:
nimages = len(image_channel_list)
assert nimages == 218673
assert len([i for i in image_channel_list.keys() if len(image_channel_list[i]) == 0]) == 0

In [8]:
conn.close()